In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!git clone https://github.com/ShaifEmon/tts_web.git

Cloning into 'tts_web'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 126 (delta 69), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (126/126), 82.95 KiB | 5.92 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install flask_ngrok
!pip install pyngrok
!pip install nest-asyncio
!pip install uvicorn==0.15.0
!pip install googletrans==4.0.0rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 6.7 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=cb5f5b6b70b1512c2e4cc72e0ff5a11333d25116370bc33c46604dc17099c8a5
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 58 kB 6.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.8 

In [ ]:
! ngrok authtoken 26keUwNK9Zgcnh6mWG7NvtFvDtM_6cKxXVfYCJCsBD1LWdam6

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext
!pip install gdown
git_repo_url = 'https://github.com/NVIDIA/tacotron2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  !cd {project_name}/waveglow && git checkout 2fd4e63
  !pip install -q librosa unidecode
  
import sys
sys.path.append(join(project_name, 'waveglow/'))
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
import gdown
d = 'https://drive.google.com/uc?id='

TensorFlow 1.x selected.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Previous HEAD position was 5bc2a53 README.md: reporting correct number after finding bug in inference time code
HEAD is now at 2fd4e63 README.md: update waveglow speed and published models url
     |████████████████████████████████| 235 kB 8.2 MB/s 


In [ ]:
force_download_TT2 = True
tacotron2_pretrained_model_sad = 'sad'
if not exists(tacotron2_pretrained_model_sad) or force_download_TT2:
               
  gdown.download(d+r'1XCHgWH855CAHcNk6sztxr95sAbWNHuRL', tacotron2_pretrained_model_sad, quiet=False); print("Tacotron2 Model Downloaded")

Downloading...
From: https://drive.google.com/uc?id=1XCHgWH855CAHcNk6sztxr95sAbWNHuRL
To: /content/sad
100%|██████████| 338M/338M [00:02<00:00, 145MB/s]

Tacotron2 Model Downloaded


In [ ]:
force_download_TT2 = True
tacotron2_pretrained_model_neutral = 'neutral'
if not exists(tacotron2_pretrained_model_neutral) or force_download_TT2:
               
  gdown.download(d+r'1ka4_Pb8QmyjX8EZiRn01jzlXgF_yE1tu', tacotron2_pretrained_model_neutral, quiet=False); print("Tacotron2 Model Downloaded")

Downloading...
From: https://drive.google.com/uc?id=1ka4_Pb8QmyjX8EZiRn01jzlXgF_yE1tu
To: /content/neutral
100%|██████████| 338M/338M [00:03<00:00, 91.6MB/s]

Tacotron2 Model Downloaded


In [ ]:
force_download_TT2 = True
tacotron2_pretrained_model_happy = 'Happy'
if not exists(tacotron2_pretrained_model_happy) or force_download_TT2:
               
  gdown.download(d+r'1LpFb_uQbCDfT3g-lb4vE9dhy61ePzcCD', tacotron2_pretrained_model_happy, quiet=False); print("Tacotron2 Model Downloaded")

Downloading...
From: https://drive.google.com/uc?id=1LpFb_uQbCDfT3g-lb4vE9dhy61ePzcCD
To: /content/Happy
100%|██████████| 338M/338M [00:31<00:00, 10.7MB/s]

Tacotron2 Model Downloaded


In [ ]:
force_download_TT2 = True
waveglow_pretrained_model = 'waveglow.pt'
if not exists(waveglow_pretrained_model):
  gdown.download(d+r'1U8eeUMB1hFIy6-nI49Jwaa1v8Qc5YfdR', waveglow_pretrained_model, quiet=False); print("WaveGlow Model Downloaded")

Downloading...
From: https://drive.google.com/uc?id=1U8eeUMB1hFIy6-nI49Jwaa1v8Qc5YfdR
To: /content/waveglow.pt
100%|██████████| 676M/676M [00:03<00:00, 189MB/s]

WaveGlow Model Downloaded


In [ ]:
%matplotlib inline
import IPython.display as ipd
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT
from audio_processing import griffin_lim
from text import text_to_sequence


graph_width = 900
graph_height = 360
def plot_data(data, figsize=(int(graph_width/100), int(graph_height/100))):
    %matplotlib inline
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none', cmap='inferno')
    fig.canvas.draw()
    plt.show()


    thisdict[(line.split(" ",1))[0]] = (line.split(" ",1))[1].strip()
def ARPA(text):
    out = ''
    for word_ in text.split(" "):
        word=word_; end_chars = ''
        while any(elem in word for elem in r"!?,.;") and len(word) > 1:
            if word[-1] == '!': end_chars = '!' + end_chars; word = word[:-1]
            if word[-1] == '?': end_chars = '?' + end_chars; word = word[:-1]
            if word[-1] == ',': end_chars = ',' + end_chars; word = word[:-1]
            if word[-1] == '.': end_chars = '.' + end_chars; word = word[:-1]
            if word[-1] == ';': end_chars = ';' + end_chars; word = word[:-1]
            else: break
        
    if out[-1] != ";": out = out + ";"
    return out

#torch.set_grad_enabled(False)

# initialize Tacotron2 with the pretrained model
hparams = create_hparams()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd /content/tacotron2

/content/tacotron2


In [ ]:
%cd /content

/content


In [ ]:
# Load WaveGlow
waveglow = torch.load(waveglow_pretrained_model)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

In [ ]:
hparams.sampling_rate = 22050 # Don't change this
hparams.max_decoder_steps = 1000 # How long the audio will be before it cuts off (1000 is about 11 seconds)
hparams.gate_threshold = 0.1 # Model must be 90% sure the clip is over before ending generation (the higher this number is, the more likely that the AI will keep generating until it reaches the Max Decoder Steps)
model1 = Tacotron2(hparams)
model1.load_state_dict(torch.load(tacotron2_pretrained_model_sad)['state_dict'])
_ = model1.cuda().eval().half()



In [ ]:
hparams.sampling_rate = 22050 # Don't change this
hparams.max_decoder_steps = 1000 # How long the audio will be before it cuts off (1000 is about 11 seconds)
hparams.gate_threshold = 0.1 # Model must be 90% sure the clip is over before ending generation (the higher this number is, the more likely that the AI will keep generating until it reaches the Max Decoder Steps)
model2 = Tacotron2(hparams)
model2.load_state_dict(torch.load(tacotron2_pretrained_model_neutral)['state_dict'])
_ = model2.cuda().eval().half()



In [ ]:
hparams.sampling_rate = 22050 # Don't change this
hparams.max_decoder_steps = 1000 # How long the audio will be before it cuts off (1000 is about 11 seconds)
hparams.gate_threshold = 0.1 # Model must be 90% sure the clip is over before ending generation (the higher this number is, the more likely that the AI will keep generating until it reaches the Max Decoder Steps)
model3 = Tacotron2(hparams)
model3.load_state_dict(torch.load(tacotron2_pretrained_model_happy)['state_dict'])
_ = model3.cuda().eval().half()



In [ ]:
%cd /content/tts_web

/content/tts_web


In [ ]:
from googletrans import Translator
translator = Translator()
ar = translator.translate('ভেবে খুব খারাপ লাগছে').text
print(ar)

Feeling very bad to think


In [ ]:
import soundfile as sf
text = """আমি তোমাকে ালোবাসি """
sigma = 1
denoise_strength = 1
raw_input = False # disables automatic ARPAbet conversion, useful for inputting your own ARPAbet pronounciations or just for testing

for i in text.split("\n"):
    if len(i) < 1: continue;
    print(i)
    if raw_input:
         if i[-1] != ";": i=i+";" 
    with torch.no_grad(): # save VRAM by not including gradients
         sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
         sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
         mel_outputs, mel_outputs_postnet, _, alignments = model2.inference(sequence)
        # plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
        #         alignments.float().data.cpu().numpy()[0].T))
         audio= waveglow.infer(mel_outputs_postnet, sigma=sigma);
         sf.write('audio.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)
         print(""); ipd.display(ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate))

    


আমি তোমাকে ালোবাসি 



In [ ]:
import soundfile as sf
import numpy as np
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template, redirect, url_for, Response


import pickle
from googletrans import Translator

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/getspeech',methods=['POST'])
def getspeech():  
    
    input =  request.form["text"]
    br=input
    translator = Translator()
    ar = translator.translate(br, dest='bn').text
    text = ar
    sigma = .8
    denoise_strength = .34
    raw_input = False # disables automatic ARPAbet conversion, useful for inputting your own ARPAbet pronounciations or just for testing
    emotion=request.form["emotion"]
    print(emotion)
    if emotion==[1]:
       for i in text.split("\n"):
         if len(i) < 1: continue;
         print(i)
         if raw_input:
             if i[-1] != ";": i=i+";" 
         with torch.no_grad(): # save VRAM by not including gradients
             sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
             sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
             mel_outputs, mel_outputs_postnet, _, alignments = model1.inference(sequence)
        # plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
        #         alignments.float().data.cpu().numpy()[0].T))
             audio= waveglow.infer(mel_outputs_postnet, sigma=sigma);
             sf.write('audio.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)
             print(""); ipd.display(ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate))
         break
    elif emotion==[2]:
       for i in text.split("\n"):
          if len(i) < 1: continue;
          print(i)
          if raw_input:
             if i[-1] != ";": i=i+";" 
          with torch.no_grad(): # save VRAM by not including gradients
             sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
             sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
             mel_outputs, mel_outputs_postnet, _, alignments = model2.inference(sequence)
        # plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
        #         alignments.float().data.cpu().numpy()[0].T))
             audio= waveglow.infer(mel_outputs_postnet, sigma=sigma);
             sf.write('audio.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)
             print(""); ipd.display(ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate))
          break
    elif emotion==[3] :
       for i in text.split("\n"):
          if len(i) < 1: continue;
          print(i)
          if raw_input:
             if i[-1] != ";": i=i+";" 
          with torch.no_grad(): # save VRAM by not including gradients
             sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
             sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
             mel_outputs, mel_outputs_postnet, _, alignments = model3.inference(sequence)
        # plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
        #         alignments.float().data.cpu().numpy()[0].T))
             audio= waveglow.infer(mel_outputs_postnet, sigma=sigma);
             sf.write('audio.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)
             print(""); ipd.display(ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate))
          break
             
    return redirect(url_for("streamwav"))
@app.route("/wav")
def streamwav():
    def generate():
        with open("/content/tts_web/audio.wav", "rb") as fwav:
            data = fwav.read(1024)
            while data:
                yield data
                data = fwav.read(1024)
    return Response(generate(), mimetype="audio/x-wav")
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://febb-34-82-172-39.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
!pip install pydub
!pip install bltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.4 MB 702 kB/s 
     |████████████████████████████████| 1.5 MB 48.9 MB/s 
     |████████████████████████████████| 749 kB 56.5 MB/s 
  Created wheel for bltk: filename=bltk-1.2-py3-none-any.whl size=17432539 sha256=88adb6e29eca1d602a55a91c1445a77336ce18228e1607e1840ef9acb9ebf924
  Stored in directory: /root/.cache/pip/wheels/0b/4f/91/e074e661b4dcbc24a83e050d1c75cecfa186ffe9d58b641c51
Successfully built bltk
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
ERROR: pip's dependency resolver does not c

In [ ]:
import soundfile as sf
import numpy as np
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template, redirect, url_for, Response
from bltk.langtools import Tokenizer
from pydub import AudioSegment
from pydub.playback import play

import pickle
from googletrans import Translator

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
    return render_template('home_page.html')


@app.route('/getspeech',methods=['POST'])
def getspeech():  
    
    input =  request.form["text"]
    br=input
    print(input)
    translator = Translator()
    ar = translator.translate(br, dest='bn').text
    text = ar
    tokenizer = Tokenizer()
    textt = tokenizer.sentence_tokenizer(text)
    print(textt)
    moon = len(textt)
    print(moon)
    sigma = .8
    denoise_strength = .4
    raw_input = False # disables automatic ARPAbet conversion, useful for inputting your own ARPAbet pronounciations or just for testing
    emotion=request.form["emotion"]
    print(emotion)
    if emotion=="sad":
        j = 0
        for j in range(moon):
            
            for i in textt[j].split("\n"):
                if len(i) < 1: continue;
                print(i)
                if raw_input:
                    if i[-1] != ";": i=i+";" 
                with torch.no_grad(): # save VRAM by not including gradients
                          sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
                          sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
                          mel_outputs, mel_outputs_postnet, _, alignments = model1.inference(sequence)
                      # plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
                      #         alignments.float().data.cpu().numpy()[0].T))
                          audio= waveglow.infer(mel_outputs_postnet, sigma=sigma);
                          
                if j == 0:
                    sf.write('audio1.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)
                    ipd.display(ipd.Audio(audio.data.cpu().numpy(), rate=hparams.sampling_rate))
                    audio1 = AudioSegment.from_wav('audio1.wav')
                    
                    
                elif j > 0: 
                      sf.write('audioo'+str(j)+'.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)   
                      
                      
                      moon2 ='audioo'+str(j)+'.wav'
                      print(moon2)
                      moon3 = 'audioo'+str(j)
                      print(moon3)
                      moon3 = AudioSegment.from_wav(moon2)
                      if j == 1:
                        combined_sounds = audio1 + moon3
                      else:
                          combined_sounds = combined_sounds + moon3
                      combined_sounds.export("/content/tts_web/audiomoon.wav", format="wav")
                      ipd.display(ipd.Audio(audio.data.cpu().numpy(), rate=hparams.sampling_rate))
                break
    elif emotion=="neutral":
        j = 0
        for j in range(moon):
            
            for i in textt[j].split("\n"):
                if len(i) < 1: continue;
                print(i)
                if raw_input:
                    if i[-1] != ";": i=i+";" 
                with torch.no_grad(): # save VRAM by not including gradients
                          sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
                          sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
                          mel_outputs, mel_outputs_postnet, _, alignments = model2.inference(sequence)
                      # plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
                      #         alignments.float().data.cpu().numpy()[0].T))
                          audio= waveglow.infer(mel_outputs_postnet, sigma=sigma);
                          
                if j == 0:
                    sf.write('audio1.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)
                    ipd.display(ipd.Audio(audio.data.cpu().numpy(), rate=hparams.sampling_rate))
                    audio1 = AudioSegment.from_wav('audio1.wav')
                    
                    
                elif j > 0: 
                      sf.write('audioo'+str(j)+'.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)   
                      
                      
                      moon2 ='audioo'+str(j)+'.wav'
                      print(moon2)
                      moon3 = 'audioo'+str(j)
                      print(moon3)
                      moon3 = AudioSegment.from_wav(moon2)
                      if j == 1:
                        combined_sounds = audio1 + moon3
                      else:
                          combined_sounds = combined_sounds + moon3
                      combined_sounds.export("/content/tts_web/audiomoon.wav", format="wav")
                      ipd.display(ipd.Audio(audio.data.cpu().numpy(), rate=hparams.sampling_rate))
                break

    elif emotion=="kobita" :
        j = 0
        for j in range(moon):
            
            for i in textt[j].split("\n"):
                if len(i) < 1: continue;
                print(i)
                if raw_input:
                    if i[-1] != ";": i=i+";" 
                with torch.no_grad(): # save VRAM by not including gradients
                          sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
                          sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
                          mel_outputs, mel_outputs_postnet, _, alignments = model4.inference(sequence)
                      # plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
                      #         alignments.float().data.cpu().numpy()[0].T))
                          audio= waveglow.infer(mel_outputs_postnet, sigma=sigma);
                          
                if j == 0:
                    sf.write('audio1.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)
                    ipd.display(ipd.Audio(audio.data.cpu().numpy(), rate=hparams.sampling_rate))
                    audio1 = AudioSegment.from_wav('audio1.wav')
                    
                    
                elif j > 0: 
                      sf.write('audioo'+str(j)+'.wav',audio[0].data.cpu().numpy().astype(np.float32),hparams.sampling_rate)   
                      
                      
                      moon2 ='audioo'+str(j)+'.wav'
                      print(moon2)
                      moon3 = 'audioo'+str(j)
                      print(moon3)
                      moon3 = AudioSegment.from_wav(moon2)
                      if j == 1:
                        combined_sounds = audio1 + moon3
                      else:
                          combined_sounds = combined_sounds + moon3
                      combined_sounds.export("/content/tts_web/audiomoon.wav", format="wav")
                      ipd.display(ipd.Audio(audio.data.cpu().numpy(), rate=hparams.sampling_rate))
                break      
    return redirect(url_for("streamwav"))
@app.route("/wav")
def streamwav():
    def generate():
        with open("/content/tts_web/audiomoon.wav", "rb") as fwav:
            data = fwav.read(1024)
            while data:
                yield data
                data = fwav.read(1024)
    return Response(generate(), mimetype="audio/x-wav")
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f428-34-82-172-39.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Jun/2022 07:26:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2022 07:26:01] "GET /favicon.ico HTTP/1.1" 404 -


i love you


127.0.0.1 - - [09/Jun/2022 07:26:05] "POST /getspeech HTTP/1.1" 400 -


[]
0


127.0.0.1 - - [09/Jun/2022 07:26:16] "GET / HTTP/1.1" 200 -
